In [ ]:
#2. epoch가 충분하지 않다고 생각해서 30 - > 100으로 증가시켰다. (1 epoch란 모든 train-set을 1번 도는 것을 의미한다.)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

CFG = {
    'IMG_SIZE':224,
    'EPOCHS':100,
    'LEARNING_RATE':5e-4,
    'BATCH_SIZE':32,
    'SEED':41
}
#3. 도배 이미지 확인 (이미지 증강을 어떤 방식으로 시키면 좋을지 고민하기위해 실제 이미지를 확인한다.

plt.figure(figsize = (15,12))
for idx, i in enumerate(train.label.unique()):
    plt.subplot(4, 7, idx+1)

    df = train[train['label'] == i].reset_index(drop = True)
    # image_path = df.loc[random.randint(0, len(df))-1, 'path']
    image_path = df.loc[random.randint(0, len(df)-1), 'img_path']
    img = Image.open(image_path)
    img = img.resize((224,224))
    plt.imshow(img)
    plt.axis('off')
    plt.title(i)
plt.tight_layout()
plt.show()

#위에 있는 번호는 클래스를 라벨 인코딩 한것을 의미하며 각각의 사진은 해당 클래스에 해당하는 하자 이미지이다.



#4. 이미지를 보고 대회 talk 게시판을 참고한 결과 이미지 증강기법 중 걸레받이수정'과 '꼬임', 그리고 '몰딩수정' 데이터를 rotate 또는 verticalflip을 하면 구분을 못하게 된다는 것을 알게되었고 이를 제외하기로 했다.

train_transforefv2 = A.Compose([
    A.RandomResizedCrop(384,384, scale=(0.2, 0.8)),
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    #A.VerticalFlip(p=0.5),  (제외)
    #A.ShiftScaleRotate(p=0.5),  (제외)
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
    A.ChannelShuffle(),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
    ])
test_transforefv2 = A.Compose([
                            A.Resize(384, 384),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
train_datasetefv2 = CustomDataset(train['img_path'].values, train['label'].values, train_transforefv2)
train_loaderefv2 = DataLoader(train_datasetefv2, batch_size = CFG['BATCH_SIZE'], sampler=sampler,shuffle=False, num_workers=0)

val_datasetefv2 = CustomDataset(val['img_path'].values, val['label'].values, test_transforefv2)
val_loaderefv2 = DataLoader(val_datasetefv2, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
#5. 클래스의 분포를 알아보았다. 결과를 확인하니 상당히 클래스가 불균등하다는 것을 알 수 있다.

df['label'].value_counts()

	훼손					1405
	오염					595
	걸레받이수정	307
	꼬임					210
	터짐					162
	곰팡이				145
	오타공				142
	몰딩수정			130
	면불량				99
	석고수정			57
	들뜸					54
	피스					51
	창틀,문틀수정	27
	울음					22
	이음부불량		17
	녹오염				14
	가구수정			12
	틈새과다			5
	반점					3
#6. 이를 해결하기 위해 부족한 클래스에 가중치를 더 부여해서 학습시 적은 클래스에 대한 학습을 중요하게 여기게 해서 부족한 클래스에 대한 학습을 높인다.

class_counts = train['label'].value_counts(sort=False).to_dict()
num_samples = sum(class_counts.values())
class_weights = {l: round(num_samples/class_counts[l], 2) for l in class_counts.keys()}

print(f'cls_cnts: {len(class_counts)}\nnum_samples:{num_samples}')
labels = train['label'].to_list()
weights = [class_weights[labels[i]] for i in range(int(num_samples))]
sampler = torch.utils.data.WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))

cls_cnts: 19
num_samples:2938
#위의 결과를 보면 클래스 수는 19개라는 것을 알 수있고 (cls_cnts: 19)

#아래를 보면 각 클래스별 가중치를 알 수 있다. 부족한 클래스에 대해 높은 가중치가 설정되었다.

class_weights

{훼손: 2.46,
 오염: 5.81,
 꼬임: 16.46,
 걸레받이수정: 11.29,
 석고수정: 60.11,
 들뜸: 64.3,
 오타공: 24.47,
 터짐: 21.43,
 곰팡이: 23.84,
 창틀,문틀수정: 125.68,
 몰딩수정: 26.59,
 면불량: 35.0,
 피스: 67.44,
 울음: 153.61,
 녹오염: 251.36,
 이음부불량: 197.5,
 반점: 1382.5,
 가구수정: 276.5,
 틈새과다: 691.25}
#7. 기존에 사용했던 cross entropy같은 경우는 모든 클래스에 대해 동일한 가중치를 가지고 손실을 계산했다면 이번에는 부족한 class에 높은 가중치를 적용해 계산하는 focal loss를 통해서 평가를 진행하겠다.

import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        if self.alpha is not None:
            alpha = self.alpha.to(inputs.device)
            focal_loss = (alpha[targets] * (1 - pt) ** self.gamma * ce_loss).mean()
        else:
            focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss
def train3(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
   #기존에는 criterion = nn.CrossEntropy()를 사용했는데 FocalLoss로 변경했다.
    criterion = FocalLoss(alpha=torch.DoubleTensor([class_weights[i] for i in range(len(class_weights))]), gamma=2).to(device)

    best_score = 0
    best_model = None

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')

        if scheduler is not None:
            scheduler.step(_val_score)

        if best_score < _val_score:
            best_score = _val_score
            best_model = model

    return best_model
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor).to(device)      # ADDED .type(torch.LongTensor)

            pred = model(imgs)

            loss = criterion(pred, labels)

            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()

            val_loss.append(loss.item())

        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')

    return _val_loss, _val_score
#8. 모델은 efficientnet을 발전시킨 EfficientNetV2M 을 사용했다. 이미지 분류에서 상당히 높은 성능을 기록한 모델이고 비교적 용량이 작기 때문에 사용했다.

class EfficientNetV2M(nn.Module):
    def __init__(self, num_classes=19, pretrained=True):
        super(EfficientNetV2M, self).__init__()
        self.backbone = timm.create_model('tf_efficientnetv2_m', pretrained=pretrained)
        self.backbone.classifier = nn.Linear(self.backbone.classifier.in_features, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        return x
modelefv2=EfficientNetV2M()
modelefv2.eval()
optimizer = torch.opt5. 5im.AdamW(params = modelefv2.parameters(), lr = 3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_modelefv2 = train3(modelefv2, optimizer, train_loaderefv2, val_loaderefv2, scheduler, device)
#9. 학습한 모델을 저장한다.

torch.save(infer_modelefv2, 'C:\\Users\\82107\\Downloads\\open (2)\\bitm_2_32_30.pt')
#10. 성능이 좋다고 알려진 efficientnet_7을 noise 데이터로 학습시킨 'tf_efficientnet_b7_ns' 모델을 이용하여 학습을  진행하였다.

       (밑의 과정은 모델명만 바꾸면 되기 때문에 생략하겠다.)

class NoisyStudentB7(nn.Module):
    def __init__(self, num_classes=19, pretrained=True):
        super(NoisyStudentB7, self).__init__()
        self.backbone = timm.create_model('tf_efficientnet_b7_ns', pretrained=True)
        self.backbone.classifier = nn.Linear(self.backbone.classifier.in_features, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        return x
#11. 또다른 모델인 vision_transformer_vit_l/16을 이용해 학습시켰다.

       (밑의 과정은 모델명만 바꾸면 되기 때문에 생략하겠다.)

class vitl(nn.Module):
    def __init__(self, num_classes=19, pretrained=True):
        super(vitl, self).__init__()
        self.backbone = timm.create_model('vit_large_patch16_384', pretrained=pretrained, num_classes=num_classes)
        self.head = nn.Linear(in_features=self.backbone.head.out_features, out_features=num_classes, bias=True)

    def forward(self, x):
        x = self.backbone(x)

        x = self.head(x)
        return x